# Classifying YouTube Comments using Supervised Learning
In this notebook, we will use supervised learning techniques to classify YouTube comments. The goal is to categorize comments into different classes such as positive, negative etc. 

Let's get started!

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 

from transformers import pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from _functions import *

## TODO : Update this file to pull in all data at once
Import the data as a dataframe.

### Data preprocessing
Clean the data to remove empty rows, row[0] containing the raw data to put the text into a form such that it is ready for BERT imput.

In [15]:
# Import the data as dataframe
data_concat = pd.DataFrame(columns=["comment"])


current_dir = Path.cwd().parent

data_path = current_dir / "Data" / "RawData" 

for file_name in os.listdir(data_path):

    # Declare the file path
    file_path = data_path / file_name

    # Import the data
    df = pd.read_csv(file_path)

    clean_comments = clean_data(df).to_frame()
    
    data_concat = pd.concat([data_concat, clean_comments])

print(data_concat.shape)

(5146, 1)


## Potentially translate here? 
Quite a few non-English comments, may be worth trying a translator. However, if it is too computationally expensive on time and memory, just drop the comments here

## Use VADER HERE
Vader is to be used for sentiment analysis at this point. 

## Use secondary classification for country/target  
Potentially use targeted BERT model here

## Use secondary classification for Individual involved  
Potentially use targeted BERT model here